In [81]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from nltk.corpus import stopwords
import string
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [24]:
#Loading the training and testing data
data=pd.read_csv('./train.csv',delimiter=",")
words_arr=np.array(data.text)
test_data=pd.read_csv('./test.csv')
x_test_1=test_data.text

In [10]:
#taking out the y_train
category=np.array(data.airline_sentiment)

In [82]:
#function to simplify the part of speech value
def getsimplepos (tag) :
    if tag.startswith( 'J'):
        return wordnet.ADJ
    elif tag.startswith( 'V'):
        return wordnet. VERB
    elif tag.startswith( 'N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet .NOUN

In [80]:
#merging the stopwords and punctuations in a variable
stops=stopwords.words('english')
punc=list(string.punctuation)
stops=stops+punc

In [97]:
#using wordNetLemmatizer
lem=WordNetLemmatizer()

In [104]:
#function to remove stopwords and lemmatize the text
def clean_words(words):
    output=[]
    for w in words:
        if(w not in stops):
            pos=pos_tag([w])
            clean_word=lem.lemmatize(w,pos=getsimplepos(pos[0][1]))
            output.append(clean_word.lower())
    return output

In [83]:
text=["".join(w) for w  in words_arr]

In [119]:
train_clean=clean_words(text)

In [120]:
test_clean=clean_words(x_test_1)

In [121]:
#using countVectorizer
cv=CountVectorizer(max_features=8000,ngram_range=(1,2))

In [122]:
#getting the data in 2d format to be used with sklearn classiferis
x_train=cv.fit_transform(train_clean)
x_test=cv.transform(test_clean)

In [123]:
clf=SVC()
clf.fit(x_train,category)

SVC()

In [126]:
y_train_pred=clf.predict(x_train)

In [124]:
y_pred=clf.predict(x_test)

In [127]:
print(classification_report(category,y_train_pred))

              precision    recall  f1-score   support

    negative       0.94      0.98      0.96      6851
     neutral       0.86      0.85      0.86      2327
    positive       0.94      0.81      0.87      1802

    accuracy                           0.92     10980
   macro avg       0.91      0.88      0.90     10980
weighted avg       0.92      0.92      0.92     10980



In [128]:
print(accuracy_score(category,y_train_pred))
print(confusion_matrix(category,y_train_pred))

0.9239526411657559
[[6695  133   23]
 [ 269 1986   72]
 [ 154  184 1464]]


In [130]:
np.savetxt('answers.csv',y_pred,fmt='%s')